<a href="https://colab.research.google.com/github/WongPuiYee/MCU/blob/master/0604OOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

我是一個開發人員，先幫我建立四個串列student_ids，names、scores和genders，分別存放使用者輸入學生的學號、學生的姓名、學生的成績、學生的性別，幫我撰寫新增、修改、刪除和查詢全部學生的學號、學生的姓名、學生的成績、學生的性別，最後寫一個新增、修改、刪除、查詢全部學生和離開的使用者介面。

我是一個開發人員，先幫我建立一個字典，該字典包含學生的學號、學生的姓名、學生的成績、學生的性別，用一個list來管理全班學生，幫我撰寫新增、修改、刪除和查詢全部學生的學號、學生的姓名、學生的成績、學生的性別，最後寫一個新增、修改、刪除、查詢全部學生和離開的使用者介面。

我是一個開發人員，先幫我建立一個學生類別，該物件包含學生的學號、學生的姓名、學生的成績、學生的性別，用一個list來管理全班學生，幫我撰寫新增、修改、刪除和查詢全部學生的學號、學生的姓名、學生的成績、學生的性別，最後寫一個新增、修改、刪除、查詢全部學生和離開的使用者介面。

In [19]:
# prompt: 我是一個開發人員，先幫我建立一個學生類別，該物件包含學生的學號、學生的姓名、學生的成績、學生的性別，用一個Sqlite database來管理全班學生，幫我撰寫新增、修改、刪除和查詢全部學生的學號、學生的姓名、學生的成績、學生的性別，最後寫一個新增、修改、刪除、查詢全部學生和離開的使用者介面。

import sqlite3

class Student:
  def __init__(self, student_id, name, score, gender):
    self.student_id = student_id
    self.name = name
    self.score = score
    self.gender = gender

class StudentDatabase:
  def __init__(self, db_name="students.db"):
    self.db_name = db_name
    self._create_table()

  def _create_table(self):
    conn = sqlite3.connect(self.db_name)
    cursor = conn.cursor()
    cursor.execute('''
      CREATE TABLE IF NOT EXISTS students (
        student_id TEXT PRIMARY KEY,
        name TEXT,
        score INTEGER,
        gender TEXT
      )
    ''')
    conn.commit()
    conn.close()

  def add_student(self, student):
    conn = sqlite3.connect(self.db_name)
    cursor = conn.cursor()
    try:
      cursor.execute("INSERT INTO students (student_id, name, score, gender) VALUES (?, ?, ?, ?)",
                     (student.student_id, student.name, student.score, student.gender))
      conn.commit()
      print("學生已成功新增。")
    except sqlite3.IntegrityError:
      print(f"錯誤：學號 {student.student_id} 已存在。")
    conn.close()

  def update_student(self, student_id, new_name=None, new_score=None, new_gender=None):
    conn = sqlite3.connect(self.db_name)
    cursor = conn.cursor()
    updates = []
    params = []
    if new_name is not None:
      updates.append("name = ?")
      params.append(new_name)
    if new_score is not None:
      updates.append("score = ?")
      params.append(new_score)
    if new_gender is not None:
      updates.append("gender = ?")
      params.append(new_gender)

    if not updates:
      print("沒有提供任何更新資料。")
      conn.close()
      return

    sql = f"UPDATE students SET {', '.join(updates)} WHERE student_id = ?"
    params.append(student_id)

    cursor.execute(sql, tuple(params))
    conn.commit()
    if cursor.rowcount > 0:
      print(f"學號 {student_id} 的學生資料已成功更新。")
    else:
      print(f"錯誤：找不到學號為 {student_id} 的學生。")
    conn.close()

  def delete_student(self, student_id):
    conn = sqlite3.connect(self.db_name)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM students WHERE student_id = ?", (student_id,))
    conn.commit()
    if cursor.rowcount > 0:
      print(f"學號 {student_id} 的學生資料已成功刪除。")
    else:
      print(f"錯誤：找不到學號為 {student_id} 的學生。")
    conn.close()

  def get_all_students(self):
    conn = sqlite3.connect(self.db_name)
    cursor = conn.cursor()
    cursor.execute("SELECT student_id, name, score, gender FROM students")
    students_data = cursor.fetchall()
    conn.close()
    return students_data

  def get_student_by_id(self, student_id):
    conn = sqlite3.connect(self.db_name)
    cursor = conn.cursor()
    cursor.execute("SELECT student_id, name, score, gender FROM students WHERE student_id = ?", (student_id,))
    student_data = cursor.fetchone()
    conn.close()
    return student_data


def display_menu():
  print("\n--- 學生管理系統 ---")
  print("1. 新增學生")
  print("2. 修改學生")
  print("3. 刪除學生")
  print("4. 查詢所有學生")
  print("5. 離開")

def main():
  db = StudentDatabase()

  while True:
    display_menu()
    choice = input("請選擇操作 (1-5): ")

    if choice == '1':
      student_id = input("請輸入學號: ")
      name = input("請輸入姓名: ")
      score = int(input("請輸入成績: "))
      gender = input("請輸入性別: ")
      student = Student(student_id, name, score, gender)
      db.add_student(student)
    elif choice == '2':
      student_id = input("請輸入要修改的學號: ")
      student_to_update = db.get_student_by_id(student_id)

      if student_to_update:
          print("\n--- 修改學生資料 ---")
          new_name = input(f"請輸入新的姓名 (目前: {student_to_update[1]}, 留空表示不修改): ") or None
          new_score_str = input(f"請輸入新的成績 (目前: {student_to_update[2]}, 留空表示不修改): ")
          new_score = int(new_score_str) if new_score_str else None
          new_gender = input(f"請輸入新的性別 (目前: {student_to_update[3]}, 留空表示不修改): ") or None
          db.update_student(student_id, new_name, new_score, new_gender)
      else:
          print(f"錯誤：找不到學號為 {student_id} 的學生。")

    elif choice == '3':
      student_id = input("請輸入要刪除的學號: ")
      db.delete_student(student_id)
    elif choice == '4':
      students = db.get_all_students()
      if students:
        print("\n--- 所有學生資料 ---")
        for student in students:
          print(f"學號: {student[0]}, 姓名: {student[1]}, 成績: {student[2]}, 性別: {student[3]}")
        print("--------------------")
      else:
        print("目前沒有學生資料。")
    elif choice == '5':
      print("程式結束。")
      break
    else:
      print("無效的選擇，請再試一次。")

if __name__ == "__main__":
  main()


--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 4

--- 所有學生資料 ---
學號: 12131484, 姓名: 黃佩怡, 成績: 88.0, 性別: 女
--------------------

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 2
請輸入要修改的學號: 1
錯誤：找不到學號為 1 的學生。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 2
請輸入要修改的學號: 12131484

--- 修改學生資料 ---
請輸入新的姓名 (目前: 黃佩怡, 留空表示不修改): py
請輸入新的成績 (目前: 88.0, 留空表示不修改): 8
請輸入新的性別 (目前: 女, 留空表示不修改): 女
學號 12131484 的學生資料已成功更新。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 4

--- 所有學生資料 ---
學號: 12131484, 姓名: py, 成績: 8.0, 性別: 女
--------------------

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 3
請輸入要刪除的學號: 12131484
學號 12131484 的學生資料已成功刪除。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 5
程式結束。


--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 4
目前沒有學生資料。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 1
請輸入學號: 9701824
請輸入姓名: lendy
請輸入成績: 90
請輸入性別: 男
學生已成功新增。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 2
請輸入要修改的學號: 001
錯誤：找不到指定的學號。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 2
請輸入要修改的學號: 9701824

--- 修改學生資料 ---
請輸入新的姓名 (目前: lendy, 留空表示不修改): lin
請輸入新的成績 (目前: 90, 留空表示不修改): 88
請輸入新的性別 (目前: 男, 留空表示不修改):
學生資料已成功更新。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 4

--- 所有學生資料 ---
學號: 9701824, 姓名: lin, 成績: 88, 性別: 男

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 3
請輸入要刪除的學號: 9701824
學生資料已成功刪除。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 4
目前沒有學生資料。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 5
程式結束。